In [1]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader
from scipy.spatial import distance

In [2]:
# input:
# new_data: dict of {book:rating,...}
# data: what was 'poc_df' -- the training data with rows like: user-booktitle-rating
# model: the surprise library object after training (what was 'algo')

# output: predictions of book which would be highest rated by the new user
# output type 
def predict(new_data, data, model):
    user_item = pd.pivot(
        data.loc[data['Book-Title'].isin(books)],
        index='User',columns='Book-Title',values='Rating')
#     scaling in order to fillna 0
    merged = user_item.append(pd.Series(new_data,name="new")).apply(lambda x: x-4.5).fillna(0)
#     compute cosine similarity
    vf = np.vectorize(lambda user: 1-distance.cosine(merged.loc[user], merged.loc['new']))
    sims = pd.Series(vf(merged.index.values[:-1]),index=user_item.index)
    get_prediction = lambda b: model.predict(uid=sims.index[sims.argmax()],iid=b).est
    preds = pd.Series(poc_df['Book-Title'].unique()).apply(get_prediction)
    df_res = pd.DataFrame(
        np.column_stack((poc_df['Book-Title'].unique(),res)),
        columns=["title","score"])
    # find largest
    df_res['score'] = df_res['score'].astype(float)
    recoms = df_res.nlargest(15,'score')
#   commands for output in the desired form (dict like)
    return dict(zip(recoms.title.to_list(),recoms.score.to_list()))
    
